In [1]:
# # Converting 
# /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00/
# /2020_01_07_at_23:28:46/classic_hl[256, 256]_dp0.4_ite3/trained_classic_model_full.pth

In [2]:
from __future__ import print_function

In [3]:
import numpy as np
import time
import os
import sys
import GPUtil
import pickle
import re

import torch

sys.path.append('/home/felix/Research/Adversarial Research/FGN---Research/')
import Finite_Gaussian_Network_lib as fgnl
import Finite_Gaussian_Network_lib.fgn_helper_lib as fgnh

In [4]:
# /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_23:28:46
# /classic_hl[64]_dp0.1_ite2/trained_classic_model_full.pth
# Current model name: converted_fgn_hl[64]_dp0.1_sphere_ord0.5_ls8e-08

In [5]:
# Preliminaries
# saved in :
base_dir =  '/home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00/'
# folder with the classic networks
exp_dir = '/2020_01_07_at_23:28:46/'

In [6]:
# load shared configs
shared_config_fixed_parameters_dic = np.load(base_dir+'/shared_config_fixed_parameters_dic.pckl',allow_pickle='TRUE')


In [7]:
batch_size = shared_config_fixed_parameters_dic['batch_size']
optimizer_name = shared_config_fixed_parameters_dic['optimizer_name']
lmbda_l2 = shared_config_fixed_parameters_dic['lmbda_l2']
opt_lr = shared_config_fixed_parameters_dic['opt_lr']
# num_epochs = shared_config_fixed_parameters_dic['num_epochs'] #not used

In [8]:
# load list of fgn configs
fgn_config_experiments_list = np.load(base_dir+'/fgn_config_experiments_list.pckl',allow_pickle='TRUE')
# same for all FGNs
non_lin = True
free_biases = True

In [9]:
classic_model_list = []
# for every trained classic model

for path, dir,files in os.walk(base_dir+exp_dir):
    for file in files:
        if 'trained_classic_model_full.pth' in file:
            
#             print('adding {}/{}'.format(path, file))
            classic_model_list.append((path,file))

In [10]:
for x in classic_model_list:
    print(x)

('/home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_23:28:46/classic_hl[16, 16, 16]_dp0.4_ite2', 'trained_classic_model_full.pth')
('/home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_23:28:46/classic_hl[256, 256]_dp0.4_ite3', 'trained_classic_model_full.pth')
('/home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_23:28:46/classic_hl[64]_dp0.1_ite2', 'trained_classic_model_full.pth')
('/home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_23:28:46/classic_hl[1024, 1024, 1024]_dp0.1_ite1', 'trained_classic_model_full.pth')
('/home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_23:28:46/classic_hl[256]_dp0.4_ite1', 'trained_classic_model_full.pth')
('/home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_23:28:46/classic_hl[64, 64]_dp0.4_ite2', 'trained_classic_

In [11]:
# first loop 
path, file = classic_model_list[0]

print('Converting {}/{}'.format(path, file))

# load config
with open(path+'/config.txt','r+') as c:
    for l in c.readlines():
        if 'hidden_layer_sizes' in l: hidden_layer_sizes = [int(x.strip(' ,][]')) for x in l[20:].split()]
        if 'lmbda_l2' in l : lmbda_l2 = float(l[8:])
        if 'drop_p' in l : drop_p = float(l[6:])

print(hidden_layer_sizes)
print(lmbda_l2)
print(drop_p)

# load classic model
classic_model = torch.load(path+'/trained_classic_model_full.pth')

# for each FGN to try
for fgn_config in fgn_config_experiments_list:
    print()

    # extract params
    covar_type = fgn_config['covar_type']
    ordinal = fgn_config['ordinal']
    lmbda_sigma = fgn_config['lmbda_sigma']

    # loss for fgn uses lmbda_sigma
    loss  = fgnl.def_fgn_cross_ent_loss(lmbda_l2=lmbda_l2, lmbda_sigs=lmbda_sigma)

    # def model name
    model_name = 'converted_fgn_hl{}_dp{}_{}_ord{}_ls{}'.format(hidden_layer_sizes, 
                                                               drop_p, 
                                                               covar_type, 
                                                               ordinal,
                                                               lmbda_sigma)

    print('Current model name: {}'.format(model_name))
    
    not_done = True
    
    if not_done:
        # create fgn model
        fgn_model = fgnl.Feedforward_FGN_net(in_feats=28*28, 
                                         out_feats=10, 
                                         hidden_layer_sizes=hidden_layer_sizes,
                                         drop_p = drop_p, 
                                         covar_type=covar_type, 
                                         ordinal=ordinal,
                                         non_lin=non_lin,
                                         free_biases=free_biases)

        print('classic MODEL')
        print(classic_model)
        print(classic_model.state_dict().keys())    
        print('FGN MODEL')
        print(fgn_model)
        print(fgn_model.state_dict().keys()) 

        # convert 
        fgnl.convert_classic_to_fgn(classic_model, fgn_model)

        # optimize every params that require grad
        if optimizer_name=='Adam':
            optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, fgn_model.parameters()), lr=opt_lr)
            
        # skip training
        print('Skip Training')
        
    # clean up GPU
    del fgn_model
    torch.cuda.empty_cache()

Converting /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_23:28:46/classic_hl[16, 16, 16]_dp0.4_ite2/trained_classic_model_full.pth
[16, 16, 16]
8e-06
0.4

Current model name: converted_fgn_hl[16, 16, 16]_dp0.4_sphere_ord0.5_ls8e-08
classic MODEL
Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Dropout(p=0.4)
    (1): Linear(in_features=784, out_features=16, bias=True)
    (2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4)
    (4): Linear(in_features=16, out_features=16, bias=True)
    (5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.4)
    (7): Linear(in_features=16, out_features=16, bias=True)
    (8): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout(p=0.4)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): Linear(i

Skip Training

Current model name: converted_fgn_hl[16, 16, 16]_dp0.4_diag_ord5.0_ls8e-07
classic MODEL
Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Dropout(p=0.4)
    (1): Linear(in_features=784, out_features=16, bias=True)
    (2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4)
    (4): Linear(in_features=16, out_features=16, bias=True)
    (5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.4)
    (7): Linear(in_features=16, out_features=16, bias=True)
    (8): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout(p=0.4)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): Linear(in_features=16, out_features=10, bias=True)
)
['hidden_layers.1.weight', 'hidden_layers.1.bias', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'hidden_layers.2.running_mean', 'hidden_l

['hidden_layers.1.weights', 'hidden_layers.1.biases', 'hidden_layers.1.centers', 'hidden_layers.1.inv_covars', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'hidden_layers.2.running_mean', 'hidden_layers.2.running_var', 'hidden_layers.2.num_batches_tracked', 'hidden_layers.4.weights', 'hidden_layers.4.biases', 'hidden_layers.4.centers', 'hidden_layers.4.inv_covars', 'hidden_layers.5.weight', 'hidden_layers.5.bias', 'hidden_layers.5.running_mean', 'hidden_layers.5.running_var', 'hidden_layers.5.num_batches_tracked', 'hidden_layers.7.weights', 'hidden_layers.7.biases', 'hidden_layers.7.centers', 'hidden_layers.7.inv_covars', 'hidden_layers.8.weight', 'hidden_layers.8.bias', 'hidden_layers.8.running_mean', 'hidden_layers.8.running_var', 'hidden_layers.8.num_batches_tracked', 'ib.weight', 'ib.bias', 'ib.running_mean', 'ib.running_var', 'ib.num_batches_tracked', 'fl.weights', 'fl.biases', 'fl.centers', 'fl.inv_covars']
Skip Training

Current model name: converted_fgn_hl[16, 16, 16]_dp0.

Skip Training

Current model name: converted_fgn_hl[16, 16, 16]_dp0.4_sphere_ord2.0_ls0.0008
classic MODEL
Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Dropout(p=0.4)
    (1): Linear(in_features=784, out_features=16, bias=True)
    (2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4)
    (4): Linear(in_features=16, out_features=16, bias=True)
    (5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.4)
    (7): Linear(in_features=16, out_features=16, bias=True)
    (8): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout(p=0.4)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): Linear(in_features=16, out_features=10, bias=True)
)
['hidden_layers.1.weight', 'hidden_layers.1.bias', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'hidden_layers.2.running_mean', 'hidde

In [12]:
# moving to the next classic model
del classic_model
torch.cuda.empty_cache()

In [13]:
# second loop 
path, file = classic_model_list[2]

print('Converting {}/{}'.format(path, file))

# load config
with open(path+'/config.txt','r+') as c:
    for l in c.readlines():
        if 'hidden_layer_sizes' in l: hidden_layer_sizes = [int(x.strip(' ,][]')) for x in l[20:].split()]
        if 'lmbda_l2' in l : lmbda_l2 = float(l[8:])
        if 'drop_p' in l : drop_p = float(l[6:])

print(hidden_layer_sizes)
print(lmbda_l2)
print(drop_p)

# load classic model
classic_model = torch.load(path+'/trained_classic_model_full.pth')

# for each FGN to try
for fgn_config in fgn_config_experiments_list:
    print()

    # extract params
    covar_type = fgn_config['covar_type']
    ordinal = fgn_config['ordinal']
    lmbda_sigma = fgn_config['lmbda_sigma']

    # loss for fgn uses lmbda_sigma
    loss  = fgnl.def_fgn_cross_ent_loss(lmbda_l2=lmbda_l2, lmbda_sigs=lmbda_sigma)

    # def model name
    model_name = 'converted_fgn_hl{}_dp{}_{}_ord{}_ls{}'.format(hidden_layer_sizes, 
                                                               drop_p, 
                                                               covar_type, 
                                                               ordinal,
                                                               lmbda_sigma)

    print('Current model name: {}'.format(model_name))
    
    not_done = True
    
    if not_done:
        # create fgn model
        fgn_model = fgnl.Feedforward_FGN_net(in_feats=28*28, 
                                         out_feats=10, 
                                         hidden_layer_sizes=hidden_layer_sizes,
                                         drop_p = drop_p, 
                                         covar_type=covar_type, 
                                         ordinal=ordinal,
                                         non_lin=non_lin,
                                         free_biases=free_biases)

        print('classic MODEL')
        print(classic_model)
        print(classic_model.state_dict().keys())    
        print('FGN MODEL')
        print(fgn_model)
        print(fgn_model.state_dict().keys()) 

        # convert 
        fgnl.convert_classic_to_fgn(classic_model, fgn_model, verbose=True)

        # optimize every params that require grad
        if optimizer_name=='Adam':
            optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, fgn_model.parameters()), lr=opt_lr)
            
        # skip training
        print('Skip Training')
        
    # clean up GPU
    del fgn_model
    torch.cuda.empty_cache()

Converting /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_23:28:46/classic_hl[64]_dp0.1_ite2/trained_classic_model_full.pth
[64]
8e-06
0.1

Current model name: converted_fgn_hl[64]_dp0.1_sphere_ord0.5_ls8e-08
classic MODEL
Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Dropout(p=0.1)
    (1): Linear(in_features=784, out_features=64, bias=True)
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.1)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): Linear(in_features=64, out_features=10, bias=True)
)
['hidden_layers.1.weight', 'hidden_layers.1.bias', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'hidden_layers.2.running_mean', 'hidden_layers.2.running_var', 'hidden_layers.2.num_batches_tracked', 'ib.weight', 'ib.bias', 'ib.running_mean', 'ib.running_var', 'ib.num_batches_tracked', 'fl.weight', 'fl.bias']
FGN MODEL
F

('NEW', ['hidden_layers.1.weights', 'hidden_layers.1.centers', 'hidden_layers.1.biases', 'hidden_layers.1.inv_covars', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'hidden_layers.2.running_mean', 'hidden_layers.2.running_var', 'hidden_layers.2.num_batches_tracked', 'ib.weight', 'ib.bias', 'ib.running_mean', 'ib.running_var', 'ib.num_batches_tracked', 'fl.weights', 'fl.centers', 'fl.biases', 'fl.inv_covars'])
Skip Training

Current model name: converted_fgn_hl[64]_dp0.1_diag_ord5.0_ls8e-08
classic MODEL
Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Dropout(p=0.1)
    (1): Linear(in_features=784, out_features=64, bias=True)
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.1)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): Linear(in_features=64, out_features=10, bias=True)
)
['hidden_layers.1.weight', 'hidden_layers.1.bias', 'hidden_layers.2.weight', 'h

()
('Classic', ['hidden_layers.1.weight', 'hidden_layers.1.bias', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'hidden_layers.2.running_mean', 'hidden_layers.2.running_var', 'hidden_layers.2.num_batches_tracked', 'ib.weight', 'ib.bias', 'ib.running_mean', 'ib.running_var', 'ib.num_batches_tracked', 'fl.weight', 'fl.bias'])
('FGN', ['hidden_layers.1.weights', 'hidden_layers.1.biases', 'hidden_layers.1.centers', 'hidden_layers.1.inv_covars', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'hidden_layers.2.running_mean', 'hidden_layers.2.running_var', 'hidden_layers.2.num_batches_tracked', 'ib.weight', 'ib.bias', 'ib.running_mean', 'ib.running_var', 'ib.num_batches_tracked', 'fl.weights', 'fl.biases', 'fl.centers', 'fl.inv_covars'])
('NEW', ['hidden_layers.1.weights', 'hidden_layers.1.centers', 'hidden_layers.1.biases', 'hidden_layers.1.inv_covars', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'hidden_layers.2.running_mean', 'hidden_layers.2.running_var', 'hidden_layers.2.num_

()
('Classic', ['hidden_layers.1.weight', 'hidden_layers.1.bias', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'hidden_layers.2.running_mean', 'hidden_layers.2.running_var', 'hidden_layers.2.num_batches_tracked', 'ib.weight', 'ib.bias', 'ib.running_mean', 'ib.running_var', 'ib.num_batches_tracked', 'fl.weight', 'fl.bias'])
('FGN', ['hidden_layers.1.weights', 'hidden_layers.1.biases', 'hidden_layers.1.centers', 'hidden_layers.1.inv_covars', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'hidden_layers.2.running_mean', 'hidden_layers.2.running_var', 'hidden_layers.2.num_batches_tracked', 'ib.weight', 'ib.bias', 'ib.running_mean', 'ib.running_var', 'ib.num_batches_tracked', 'fl.weights', 'fl.biases', 'fl.centers', 'fl.inv_covars'])
('NEW', ['hidden_layers.1.weights', 'hidden_layers.1.centers', 'hidden_layers.1.biases', 'hidden_layers.1.inv_covars', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'hidden_layers.2.running_mean', 'hidden_layers.2.running_var', 'hidden_layers.2.num_

('NEW', ['hidden_layers.1.weights', 'hidden_layers.1.centers', 'hidden_layers.1.biases', 'hidden_layers.1.inv_covars', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'hidden_layers.2.running_mean', 'hidden_layers.2.running_var', 'hidden_layers.2.num_batches_tracked', 'ib.weight', 'ib.bias', 'ib.running_mean', 'ib.running_var', 'ib.num_batches_tracked', 'fl.weights', 'fl.centers', 'fl.biases', 'fl.inv_covars'])
Skip Training

Current model name: converted_fgn_hl[64]_dp0.1_diag_ord0.5_ls1.6e-05
classic MODEL
Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Dropout(p=0.1)
    (1): Linear(in_features=784, out_features=64, bias=True)
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.1)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): Linear(in_features=64, out_features=10, bias=True)
)
['hidden_layers.1.weight', 'hidden_layers.1.bias', 'hidden_layers.2.weight', 

classic MODEL
Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Dropout(p=0.1)
    (1): Linear(in_features=784, out_features=64, bias=True)
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.1)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): Linear(in_features=64, out_features=10, bias=True)
)
['hidden_layers.1.weight', 'hidden_layers.1.bias', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'hidden_layers.2.running_mean', 'hidden_layers.2.running_var', 'hidden_layers.2.num_batches_tracked', 'ib.weight', 'ib.bias', 'ib.running_mean', 'ib.running_var', 'ib.num_batches_tracked', 'fl.weight', 'fl.bias']
FGN MODEL
Feedforward_FGN_net(
  (hidden_layers): ModuleList(
    (0): Dropout(p=0.1)
    (1): FGN_layer()
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.1)
  )
  (ib): BatchNorm1d(784, eps=1e-05, mome

In [14]:
classic_model.state_dict().keys()

['hidden_layers.1.weight',
 'hidden_layers.1.bias',
 'hidden_layers.2.weight',
 'hidden_layers.2.bias',
 'hidden_layers.2.running_mean',
 'hidden_layers.2.running_var',
 'hidden_layers.2.num_batches_tracked',
 'ib.weight',
 'ib.bias',
 'ib.running_mean',
 'ib.running_var',
 'ib.num_batches_tracked',
 'fl.weight',
 'fl.bias']

In [15]:
fgn_model.state_dict().keys()

NameError: name 'fgn_model' is not defined

In [ ]:
fgnl.convert_classic_to_fgn(classic_model, fgn_model)
